In [ ]:
import os
import numpy as np
import nibabel as nib
import cv2
from tqdm import tqdm

# Extract NifTI dataset

In [ ]:
# Directories
input_dir = "D:/coding/SKRIPSI/CODE/ImageCHD_dataset"
output_image_dir = "D:/coding/SKRIPSI/CODE/yolo_data/images"
output_label_dir = "D:/coding/SKRIPSI/CODE/yolo_data/labels"

# Ensure output directories exist
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Get all image files
nii_files = sorted([f for f in os.listdir(input_dir) if f.endswith("_image.nii.gz")])

# Function to convert NIfTI to JPG
def convert_nifti_to_jpg(nifti_path, output_dir, is_label=False):
    # Load NIfTI file
    nifti_img = nib.load(nifti_path)
    image_data = nifti_img.get_fdata()

    # Normalize image (for visualization)
    if not is_label:
        image_data = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data)) * 255

    # Convert each slice to JPG
    filename = os.path.basename(nifti_path).replace("_image.nii.gz", "").replace("_label.nii.gz", "")
    for i in range(image_data.shape[2]):  # Iterate over slices
        slice_img = image_data[:, :, i]

        # Convert label to binary (thresholding)
        if is_label:
            slice_img = (slice_img > 0).astype(np.uint8) * 255  

        # Save as JPG
        output_path = os.path.join(output_dir, f"{filename}_{i:03d}.jpg")
        cv2.imwrite(output_path, slice_img)

# Process all images
print("🔄 Converting groundtruth images...")
for nii_file in tqdm(nii_files):
    convert_nifti_to_jpg(os.path.join(input_dir, nii_file), output_image_dir, is_label=False)

# Process all labels
print("🔄 Converting label masks (binary)...")
label_files = sorted([f.replace("_image.nii.gz", "_label.nii.gz") for f in nii_files])
for label_file in tqdm(label_files):
    convert_nifti_to_jpg(os.path.join(input_dir, label_file), output_label_dir, is_label=True)

print("✅ Conversion complete! All images & labels are saved in YOLO format.")
